In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!pip install en_core_web_lg

In [ ]:
import string
import re
import os
import nltk

#nltk.download('stopwords')
#from nltk.tokenize import TweetTokenizer
#from nltk.corpus import stopwords
#from nltk.stem import PorterStemmer
import spacy

import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt

import tensorflow as tf

In [ ]:
lg_nlp = spacy.load("en_core_web_lg")

In [ ]:
train_df= pd.read_csv("/kaggle/input/nlp-getting-started/train.csv")
test_df= pd.read_csv("/kaggle/input/nlp-getting-started/test.csv")
sample_sub_df= pd.read_csv("/kaggle/input/nlp-getting-started/sample_submission.csv")

train_val_tweets = train_df.text.values.tolist()
test_tweets = test_df.text.values.tolist()
train_val_labels = train_df.target.values

from sklearn.model_selection import train_test_split
train_tweets, val_tweets, y_train, y_val = train_test_split(train_val_tweets, train_val_labels, test_size=0.10, random_state=42, shuffle=True)
np.unique(y_train, return_counts=True)[1], np.unique(y_val, return_counts=True)[1]

#  Data preparation

##  text cleaning

In [ ]:
class MyTweetTokenizer:

    def __init__(self, num_tokens=10000, do_lower=True, remove_stop_words = True, 
                 replace_nb_with_fix_tok=True, oov_token='_OOV_', stop_words = [], use_lemma=True, nlp_model = None):
        self.num_tokens = num_tokens
        self.do_lower = do_lower        
        self.index_token = {0: '_PAD_', 1: oov_token}
        self.token_index = {'_PAD_': 0, oov_token: 1}        
        self.token_count = {'_PAD_':0, oov_token: 0}
        self.oov_token = oov_token

        self.replace_nb_with_fix_tok = replace_nb_with_fix_tok
        #self.tokenizer = TweetTokenizer(preserve_case=False, strip_handles=True, reduce_len=True)
        #self.stopwords_english = stopwords.words('english')
        #self.stemmer = PorterStemmer()
        self.DIGIT_TOKEN = '_DIGIT_'
        self.URL_TOKEN = '_URL_'
        self.EMAIL_TOKEN = '_EMAIL_'
        self.use_lemma = use_lemma
        if nlp_model==None:
            self.nlp = spacy.load("en_core_web_sm", disable=['tagger','parser','ner'])
        else:
            self.nlp = nlp_model
        
        self.remove_stop_words = remove_stop_words        
        self.stop_words = stop_words
        if len(stop_words)>0:
            print("custom stop words")
            self.nlp.Defaults.stop_words=self.stop_words

    def __add_token(self, token, i): 
        self.index_token[i]=token
        self.token_index[token]=i
        count = self.token_count.get(token,0)
        self.token_count[token] = count+1
        
        
    def __clean_emoji(self, tweet):        
        # have many categories
        tweet = re.compile("["u"\U0001F600-\U0001F64F""]+", flags=re.UNICODE).sub(r'__emoticon__', tweet)
        tweet = re.compile("["u"\U0001F300-\U0001F5FF""]+", flags=re.UNICODE).sub(r'__pictograph__', tweet)
        tweet = re.compile("["u"\U0001F680-\U0001F6FF""]+", flags=re.UNICODE).sub(r'__transport__', tweet)
        tweet = re.compile("["u"\U0001F1E0-\U0001F1FF""]+", flags=re.UNICODE).sub(r'__flag__', tweet)
        tweet = re.compile("["u"\U00002702-\U000027B0"
                              u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE).sub(r'__other_emoji__', tweet)
        
        '''
        emoji_pattern = re.compile("["
                                   u"\U0001F600-\U0001F64F"  # emoticons
                                   u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                                   u"\U0001F680-\U0001F6FF"  # transport & map symbols
                                   u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                                   u"\U00002702-\U000027B0"
                                   u"\U000024C2-\U0001F251"
                                   "]+", flags=re.UNICODE)
        tweet = emoji_pattern.sub(r'__emoji__', tweet)
        '''        
        return tweet
        

    def __process_tweet(self, tweet):   
        # remove emoji -> we keep emoji cause they have meaning
        tweet = self.__clean_emoji(tweet)
        
        if self.do_lower:
            tweet = tweet.lower()
        # remove old style retweet text "RT"
        tweet = re.sub(r'^RT[\s]+', '', tweet)
        # remove hashtags
        # only removing the hash # sign from the word
        tweet = re.sub(r'#', '', tweet)
        # remove html
        tweet = re.sub(r'<.*?>|&([a-z0-9]+|#[0-9]{1,6}|#x[0-9a-f]{1,6});', '', tweet)        
        # remove special chars 
        # repr to convert to raw string in order to handle this case
        tweet = re.sub(r"\\x\d\d\w\w",'', repr(tweet))
        
        return tweet
    
    
    def __process_token_type(self, token):
        if token.is_digit:
            return_token = self.DIGIT_TOKEN
        elif token.like_url:                 
            return_token = self.URL_TOKEN                
        elif token.like_email:
            return_token = self.EMAIL_TOKEN
        else:
            return_token = token.lemma_
        return return_token
    
    def __process_token(self, token):
        return_token = None     
        
        
        if self.remove_stop_words:
            if not token.is_stop:
                return_token = self.__process_token_type(token)
        else:
            return_token = self.__process_token_type(token)
                
        return return_token
        
        
        
    def fit_on_texts(self, tweets):
        
        i = len(self.index_token)  # we start after already entered tokens
        for tweet in tweets:
            tweet = self.__process_tweet(tweet)
            # tokenize tweets
            #print("pre-tokenizer:",tweet)
            tweet_tokens = self.nlp(tweet) #self.tokenizer.tokenize(tweet)    
            
            for tok in tweet_tokens:
                #print(">",tok)
                tok = self.__process_token(tok)
                if tok is not None:
                    #print(">",tok, type(tok))
                    # tok can already be in index. If true we don't add it, but reuse existing idx
                    idx = self.token_index.get(tok, i)
                    self.__add_token(tok,idx)
                    if (idx==i):
                        i+=1
 
        ## set token counts of OOV and PAD to vocab size so they are keyps
        # eliminate words over num_words based on sort of count.
        top_tokens = [k for k,v in sorted(list(self.token_count.items()), key=lambda item: item[1], reverse=True)]
        top_tokens = top_tokens[:self.num_tokens-2] 
        top_token_index = {'_PAD_': 0, self.oov_token: 1} 
        
        self.token_index = {**top_token_index, 
                            **{t: i+2 for i, t in enumerate(top_tokens)}}
        self.index_token = {v : k for (k, v) in self.token_index.items()}
        self.top_tokens = {*top_tokens}        

        
    def __texts_to_sequences(self, tweets, with_token_counts=False):
        counters = {}
        seqs = []
        for tweet in tweets:
            tweet = self.__process_tweet(tweet)
            tweet_tokens = self.nlp(tweet)
            seq = []
            for tok in tweet_tokens:
                tok = self.__process_token(tok) 
                
                if tok is not None:                    
                    if tok in self.top_tokens:
                        #print("inv",tok)
                        tok_idx = self.token_index[tok]
                    else:                        
                        tok_idx = self.token_index[self.oov_token]
                    
                    tok_count = counters.get(tok_idx,0)
                    counters[tok_idx]=tok_count+1
                    
                    seq.append(tok_idx)
            seqs.append(seq)
        return seqs, counters
    
                
    def texts_to_sequences(self, tweets):
        
        seqs, _ = self.__texts_to_sequences(tweets, with_token_counts=False)
        
        return seqs
    
    def texts_to_token_count(self, tweets):        
        seqs, counters = self.__texts_to_sequences(tweets, with_token_counts=False)
        return counters
    
    
    def sequences_to_texts(self, seqs):
        texts = []
        for seq in seqs:
            text = ' '.join([self.index_token.get(tok_idx,'') for tok_idx in seq])            
            texts.append(text)
        return texts


In [ ]:
sm_tt = MyTweetTokenizer(num_tokens=25, nlp_model= lg_nlp, use_lemma=True, remove_stop_words=False)
sm_tweets = ["the dog",
                "this is a new 2000 @jk http://www.news.com \U0001F600 !- johndoe@acme.org“) ",
               "#go https://www.ls.ch:80/home?lan=fr long life \x89ÛÒ to sport in 2021 #football HOP LS not me !-)) ?!!!"]
sm_tt.fit_on_texts(sm_tweets)
#seqs = 
#tt.texts_to_sequences(test_tweets)
#tt.sequences_to_texts(seqs)
sm_tt.token_index, sm_tt.index_token
#tt.sequences_to_texts(tt.texts_to_sequences(test_tweets)), 
sm_tt.texts_to_sequences(sm_tweets), sm_tt.token_index['_OOV_'], sm_tt.texts_to_token_count(sm_tweets)

In [ ]:
# tokenizer for train/test
VOCAB_SIZE = 30000
tweet_tok = MyTweetTokenizer(VOCAB_SIZE, remove_stop_words=False)
tweet_tok.fit_on_texts(train_tweets)

In [ ]:
# check that tweet with straing escape char (like \x89ÛÒ) are cleaned correctly
print(train_df[train_df.id==56].text.values.tolist())
toks = tweet_tok.texts_to_sequences(train_df[train_df.id==56].text.values.tolist())
tweet_tok.sequences_to_texts(toks)


In [ ]:
train_seq = tweet_tok.texts_to_sequences(train_tweets)
val_seq = tweet_tok.texts_to_sequences(val_tweets)
test_seq = tweet_tok.texts_to_sequences(test_tweets)


In [ ]:
# len of 40 captures most of tweets
SEQ_LEN=40
train_seq_pad = tf.keras.preprocessing.sequence.pad_sequences(train_seq,maxlen=SEQ_LEN,padding='post') 
val_seq_pad = tf.keras.preprocessing.sequence.pad_sequences(val_seq,maxlen=SEQ_LEN,padding='post')
test_seq_pad = tf.keras.preprocessing.sequence.pad_sequences(test_seq,maxlen=SEQ_LEN,padding='post')

X_test = test_seq_pad
X_val = val_seq_pad
X_train = train_seq_pad


In [ ]:
test_tok_counts = tweet_tok.texts_to_token_count(test_tweets)
val_tok_counts = tweet_tok.texts_to_token_count(val_tweets)
train_tok_counts = tweet_tok.texts_to_token_count(train_tweets)

In [ ]:
test_tok_counts
oov_tok_idx = tweet_tok.token_index['_OOV_']
tweet_tok.index_token[oov_tok_idx]

"train oov", train_tok_counts.get(oov_tok_idx),"val oov", val_tok_counts.get(oov_tok_idx), "test oov", test_tok_counts.get(oov_tok_idx)


### Compute tweet len

In [ ]:
train_max_len = max([len(tweet) for tweet in train_seq])
train_max_len

In [ ]:
plt.hist(np.array([len(tweet) for tweet in train_seq]))

## Split train into train and xval

In [ ]:
train_df[train_df.target==1].target.count(), train_df[train_df.target==0].target.count()

In [ ]:
train_ds = tf.data.Dataset.from_tensor_slices((X_train, y_train)).shuffle(X_train.shape[0])
val_ds = tf.data.Dataset.from_tensor_slices((X_val, y_val))
test_ds = tf.data.Dataset.from_tensor_slices((test_seq_pad))

## Model

In [ ]:
import tensorflow as tf
import tensorflow_addons as tfa
tf.__version__

### compute initial bias 

In [ ]:
neg, pos = np.bincount(train_df.target)
total = neg + pos
initial_bias = np.log([pos/neg])
initial_bias, pos/neg

In [ ]:
THRESHOLD=0.5
# use f1 for evaluation
METRICS = [        
    tfa.metrics.F1Score(num_classes=2, average='micro', threshold=THRESHOLD)    
]

def make_rnn_model(seq_len, vocab_size, hidden_size, nb_recurrent_layers=1, is_bidirectional=True, metrics=None, output_bias=None, lr=1e-3, do_dropout=False, do_batch_norm=False):
    if output_bias is not None:
        output_bias = tf.keras.initializers.Constant(output_bias)

    if do_dropout==True:        
        dropout=0.5
    else:
        dropout=0.
    inputs = tf.keras.layers.Input(shape=(seq_len,))    # use mask_zero=True for variable sequence len
    x = tf.keras.layers.Embedding(vocab_size, hidden_size, mask_zero=True)(inputs) 
    
    for i in range(nb_recurrent_layers): 
        if i<nb_recurrent_layers-1:
            do_return_seq = True
        else:
            do_return_seq = False
        if is_bidirectional:
            x = tf.keras.layers.Bidirectional(tf.keras.layers.GRU(hidden_size, dropout=dropout,recurrent_dropout=dropout, return_sequences=do_return_seq))(x)
        else:
            x = tf.keras.layers.GRU(hidden_size, dropout=dropout,recurrent_dropout=dropout, return_sequences=do_return_seq)(x)
        if do_batch_norm:
            x = tf.keras.layers.BatchNormalization()(x)    
    
    x = tf.keras.layers.Dense(hidden_size, activation='relu')(x)
    if do_dropout:
        x = tf.keras.layers.Dropout(dropout)(x)
    if do_batch_norm:
        x = tf.keras.layers.BatchNormalization()(x)

    x = tf.keras.layers.Dense(hidden_size/2, activation='relu')(x)
    if do_dropout:
        x = tf.keras.layers.Dropout(dropout)(x)
    if do_batch_norm:
        x = tf.keras.layers.BatchNormalization()(x)
        
    outputs= tf.keras.layers.Dense(1, activation='sigmoid', bias_initializer=output_bias)(x)
    
    #x=  tf.keras.layers.GRU(hidden_size, )(x)
    #x = tf.keras.layers.Dense(hidden_size, activation='relu')(x)
    ## init bias in final layer correctly. We initialize the bias to our prior knowledge 
    #outputs = tf.keras.layers.Dense(1, activation='sigmoid', bias_initializer=output_bias)(x)
    
    model = tf.keras.Model(inputs=inputs, outputs=outputs, name="rnn")    
        
    #0.0005
    model.compile(optimizer=tf.keras.optimizers.Adam(lr=lr),
                  loss=tf.keras.losses.BinaryCrossentropy(),
                  metrics=metrics)
    return model

make_rnn_model(SEQ_LEN, VOCAB_SIZE, 64, nb_recurrent_layers=1, is_bidirectional=True, metrics=METRICS, output_bias=initial_bias).summary()


### ensure model fit to a small dataset

try with a dataset of 2 and 10. It should perfecBinaryAccuracyBatchNormalization

In [ ]:
do_small_data_fit = True

if do_small_data_fit:
    def create_small_data(X,y, nb_per_class=1):
        true_idxs = np.where(y == True)[0][:nb_per_class]
        false_idxs = np.where(y == False)[0][:nb_per_class]

        idxs = np.concatenate((true_idxs, false_idxs)).tolist()    
        X_sm = X[idxs]
        y_sm = y[idxs]
        return X_sm, y_sm

    # try with 2 and 10 samples
    SM_NB=10
    X_train_sm, y_train_sm = create_small_data(X_train, y_train, nb_per_class=int(SM_NB/2))


    sm_model = make_rnn_model(SEQ_LEN, VOCAB_SIZE, 64, nb_recurrent_layers=2, is_bidirectional=False, do_dropout=False, do_batch_norm=False, metrics=METRICS, output_bias=initial_bias)
    sm_history = sm_model.fit(
        X_train_sm,
        y_train_sm,
        batch_size=SM_NB,
        epochs=100,
        verbose=0)

    threshold = 0.5
    sm_pred_prob = sm_model.predict(X_train_sm)
    sm_pred = np.where(sm_pred_prob > THRESHOLD, 1,0)
    print("pred probabilities: ")
    print(sm_pred_prob)
    print("pred:")
    print(sm_pred)


In [ ]:
mpl.rcParams['figure.figsize'] = (8, 6)
colors = plt.rcParams['axes.prop_cycle'].by_key()['color']

def plot_metric(history, label, metric, n):
# Use a log scale on y-axis to show the wide range of values.
  plt.plot(history.epoch, history.history[metric],
               color=colors[n], label='Train ' + label)
  plt.plot(history.epoch, history.history['val_'+metric],
               color=colors[n], label='Val ' + label,
               linestyle="--")
  plt.legend()
  plt.xlabel('Epoch')
  plt.ylabel(metric)



In [ ]:
tf.config.list_physical_devices('GPU')


## train and test model

In [ ]:
import datetime
EPOCHS = 10
BATCH_SIZE = 64
do_regularize = True
if do_regularize:
    cbs = [tf.keras.callbacks.EarlyStopping(monitor='loss', patience=3, verbose=1),
          tf.keras.callbacks.ReduceLROnPlateau(monitor='loss', factor=0.2, patience=3, min_lr=0.001,verbose=1)]
else:
    cbs = [tf.keras.callbacks.ReduceLROnPlateau(monitor='loss', factor=0.2, patience=3, min_lr=0.001,verbose=1)]

In [ ]:
rnn_model = make_rnn_model(SEQ_LEN, VOCAB_SIZE, 64,  nb_recurrent_layers=1 , is_bidirectional=False, metrics=METRICS, output_bias=initial_bias, lr=3e-4, do_dropout=True, do_batch_norm=True)
rnn_history = rnn_model.fit(
        X_train,
        y_train,
        batch_size=BATCH_SIZE,
        epochs=20,
        validation_data=(X_val, y_val),
        callbacks=cbs,
        verbose=1)
    
plot_metric(rnn_history, "RNN",'loss', 0)

In [ ]:
#### log_dir="logs/rnn-model/" + datetime.datetime.now().strftime("%Y.%m.%d-%H.%M")
#tensorboard_cb = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1, write_graph=True, embeddings_freq=1)
#cbs.append(tensorboard_cb)

rnn_model = make_rnn_model(SEQ_LEN, VOCAB_SIZE, 64,  nb_recurrent_layers=2 , is_bidirectional=False, metrics=METRICS, output_bias=initial_bias, lr=3e-4, do_dropout=True, do_batch_norm=True)
rnn_history = rnn_model.fit(
        X_train,
        y_train,
        batch_size=BATCH_SIZE,
        epochs=20,
        validation_data=(X_val, y_val),
        callbacks=cbs,
        verbose=1)
    
plot_metric(rnn_history, "RNN",'loss', 0)

plot_metric(rnn_history, "RNN",'f1_score', 0)

In [ ]:
#check if lot of oov in val_X
VOCAB_SIZE

### Model evaluation

| Model | f1 train | f1 val | Status |
| :-----|:--------:|:------:|:------:|
| seq_len=40, vocab=20k, hidden=32, nb_rec_layers=1, bidi=True, drop=True, batch_norm=True, epoch=20, lr=3e-4 | 0.89 | 0.74 | ok |
| seq_len=40, vocab=20k,  hidden=32, nb_rec_layers=2, bidi=True, drop=True, batch_norm=True, epoch=20, lr=3e-4 | 0.89 | 0.75 | ok |
| seq_len=40, vocab=20k,  hidden=16, nb_rec_layers=1, bidi=True, drop=True, batch_norm=True, epoch=20, lr=3e-4 | 0.74 | 0.68 | ok |
| seq_len=40, vocab=20k,  hidden=16, nb_rec_layers=2, bidi=True, drop=True, batch_norm=True, epoch=20, lr=3e-4 | 0.51 | 0.69 | ok |
| seq_len=40, vocab=20k,  hidden=64, nb_rec_layers=1, bidi=True, drop=True, batch_norm=True, epoch=20, lr=3e-4 | 0.94 | 0.75 | ok |
| seq_len=40, vocab=20k,  hidden=64, nb_rec_layers=2, bidi=True, drop=True, batch_norm=True, epoch=20, lr=3e-4 | 0.96 | 0.74 | ok |
| seq_len=40, vocab=20k, hidden=32, nb_rec_layers=1,  bidi=False, drop=True, batch_norm=True, epoch=20, lr=3e-4 | 0. | 0. |  |
| seq_len=40, vocab=20k,  hidden=32, nb_rec_layers=2, bidi=False, drop=True, batch_norm=True, epoch=20, lr=3e-4 | 0. | 0. |  |
| seq_len=40, vocab=20k,  hidden=16, nb_rec_layers=1, bidi=False, drop=True, batch_norm=True, epoch=20, lr=3e-4 | 0. | 0. |  |
| seq_len=40, vocab=20k,  hidden=16, nb_rec_layers=2, bidi=False, drop=True, batch_norm=True, epoch=20, lr=3e-4 | 0. | 0. |  |
| seq_len=40, vocab=20k,  hidden=64, nb_rec_layers=1, bidi=False, drop=True, batch_norm=True, epoch=20, lr=3e-4 |0.95. | 0.7297 | ok |
| seq_len=40, vocab=20k,  hidden=64, nb_rec_layers=2, bidi=False, drop=True, batch_norm=True, epoch=20, lr=3e-4 |0.9410| 0.7280 | ok |
| seq_len=40, vocab=30k,  hidden=64, nb_rec_layers=1, bidi=False, drop=True, batch_norm=True, epoch=20, lr=3e-4 |0.9491 | 0.7230 | ok |
| seq_len=40, vocab=30k,  hidden=64, nb_rec_layers=2, bidi=False, drop=True, batch_norm=True, epoch=20, lr=3e-4 |0.9427| 0.7171 | ok |



In [ ]:
test_pred_prob = rnn_model.predict(X_test)
threshold = 0.5
test_pred = np.where(test_pred_prob > threshold, 1,0)
test_pred = np.squeeze(test_pred)

In [ ]:
zip((test_df.id.values, test_pred))

test_ids = test_df.id.values
test_pred_df = pd.DataFrame({'id':test_ids, 'target':test_pred})
test_pred_df.to_csv("submission.csv", index=False)
#test_ids.shape, test_pred.shape

### Test the impact of initial bias initialization

In [ ]:
from datetime import datetime
EPOCHS = 10
BATCH_SIZE = 64
do_analyze_bias_init = False
if do_analyze_bias_init:
    zero_bias_model = make_model(SEQ_LEN, VOCAB_SIZE, 64, METRICS)
    zero_bias_model.layers[-1].bias.assign([0.0])
    print("zeros bias")
    zero_bias_history = zero_bias_model.fit(
        X_train,
        y_train,
        batch_size=BATCH_SIZE,
        epochs=EPOCHS,    
        validation_data=(X_val, y_val), 
        verbose=1)
    print("careful bias")
    care_bias_model = make_model(SEQ_LEN, VOCAB_SIZE, 64, METRICS, initial_bias)
    careful_bias_history = care_bias_model.fit(
        X_train,
        y_train,
        batch_size=BATCH_SIZE,
        epochs=EPOCHS,
        validation_data=(X_val, y_val),
        verbose=1)
    
    plot_metric(zero_bias_history, "Zero Bias",'loss', 0)
    plot_metric(careful_bias_history, "Careful Bias",'loss', 1)    

In [ ]:
X_train.max(), 